In [2]:
import os

import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass

In [3]:
time_to = int(datetime.now().timestamp())
time_from = int((time_to - timedelta(days=120).total_seconds()))
time_from, time_to

(1702154959, 1712522959)

In [ ]:
resp = requests.get('https://api.kraken.com/0/public/OHLC?pair=WIFUSD&interval=60&since=1701891170')

data = resp.json()['result']['WIFUSD']


In [ ]:
pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count']).tail()

In [8]:
def read_ohlcv_date(symbol: str) -> pd.DataFrame:
    time_to = int(datetime.now().timestamp())
    time_from = int((time_to - timedelta(days=120).total_seconds()))
    resp = requests.get(f'https://api.kraken.com/0/public/OHLC?pair={symbol.upper()}USD&interval=60&since={time_from}')
    print(resp.status_code)
    print(resp.json())
    data = resp.json()['result'][f'{symbol.upper()}USD']
    df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count'])
    df.drop(columns=['vwap', 'count'], inplace=True)
    df.rename(columns={'time': 'date'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    df.set_index('date', inplace=True)
    df = df.iloc[:-1]
    return df

In [9]:
sol_df = read_ohlcv_date('boden')
sol_df.tail()

200
{'error': ['EQuery:Unknown asset pair']}


KeyError: 'result'

In [ ]:
sol_df.dtypes

In [ ]:
sol_df.ta.macd(append=True)
sol_df.ta.rsi(append=True)
sol_df.ta.bbands(append=True)
sol_df.ta.obv(append=True)

sol_df.ta.sma(length=20, append=True)
sol_df.ta.ema(length=50, append=True)
sol_df.ta.stoch(append=True)
sol_df.ta.adx(append=True)

sol_df.ta.willr(append=True)
sol_df.ta.cmf(append=True)
sol_df.ta.psar(append=True)

In [ ]:
sol_df.tail()

In [ ]:
sol_df['OBV_in_million'] = sol_df['OBV']/1e7
sol_df['MACD_histogram_12_26_9'] = sol_df['MACDh_12_26_9']

In [ ]:
last_day_summary = sol_df.iloc[-1][['close',
    'MACD_12_26_9','MACD_histogram_12_26_9', 'RSI_14', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0','SMA_20', 'EMA_50','OBV_in_million', 'STOCHk_14_3_3', 
    'STOCHd_14_3_3', 'ADX_14',  'WILLR_14', 'CMF_20', 
    'PSARl_0.02_0.2', 'PSARs_0.02_0.2'
]]

print("Summary of Technical Indicators for the Last Day:")
print(last_day_summary)

In [ ]:
OPENAI_API_KEY = getpass("Enter your OpenAI API Key: ")

In [ ]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
# llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Assume the role as a leading Technical Analysis (TA) expert in the stock market, \
a modern counterpart to Charles Dow, John Bollinger, and Alan Andrews. \
Your mastery encompasses both stock fundamentals and intricate technical indicators. \
You possess the ability to decode complex market dynamics, \
providing clear insights and recommendations backed by a thorough understanding of interrelated factors. \
Your expertise extends to practical tools like the pandas_ta module, \
allowing you to navigate data intricacies with ease. \
As a TA authority, your role is to decipher market trends, make informed predictions, and offer valuable perspectives.

given {symbol} TA data as below on the last trading day, what will be the next few days possible crypto price movement? 

Summary of Technical Indicators for the Last Day:
{last_day_summary}""".format(symbol="WIF", last_day_summary=last_day_summary))

In [ ]:
chain = prompt | llm | StrOutputParser()

In [ ]:
response = chain.invoke({})

In [ ]:
print(response)